[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/MagicTime-jupyter/blob/main/MagicTime_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/MagicTime
%cd /content/MagicTime

!pip install -q xformers==0.0.25 einops omegaconf gradio accelerate==0.28.0 diffusers==0.11.1 transformers==4.38.2 jax==0.4.19 jaxlib==0.4.19 ms-swift
!apt -y install -qq aria2
!git clone -b fp16 https://huggingface.co/runwayml/stable-diffusion-v1-5 /content/MagicTime/ckpts/Base_Model/stable-diffusion-v1-5
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/resolve/main/ckpts/Base_Model/motion_module/motion_module.ckpt -d /content/MagicTime/ckpts/Base_Model/motion_module -o motion_module.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/resolve/main/ckpts/DreamBooth/RcnzCartoon.safetensors -d /content/MagicTime/ckpts/DreamBooth -o RcnzCartoon.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/resolve/main/ckpts/DreamBooth/RealisticVisionV60B1_v51VAE.safetensors -d /content/MagicTime/ckpts/DreamBooth -o RealisticVisionV60B1_v51VAE.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/resolve/main/ckpts/DreamBooth/ToonYou_beta6.safetensors -d /content/MagicTime/ckpts/DreamBooth -o ToonYou_beta6.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/resolve/main/ckpts/Magic_Weights/magic_adapter_s/magic_adapter_s.ckpt -d /content/MagicTime/ckpts/Magic_Weights/magic_adapter_s -o magic_adapter_s.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/raw/main/ckpts/Magic_Weights/magic_adapter_t/configuration.json -d /content/MagicTime/ckpts/Magic_Weights/magic_adapter_t -o configuration.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/raw/main/ckpts/Magic_Weights/magic_adapter_t/default/adapter_config.json -d /content/MagicTime/ckpts/Magic_Weights/magic_adapter_t/default -o adapter_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/resolve/main/ckpts/Magic_Weights/magic_adapter_t/default/adapter_model.bin -d /content/MagicTime/ckpts/Magic_Weights/magic_adapter_t/default -o adapter_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/raw/main/ckpts/Magic_Weights/magic_text_encoder/configuration.json -d /content/MagicTime/ckpts/Magic_Weights/magic_text_encoder -o configuration.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/resolve/main/ckpts/Magic_Weights/magic_text_encoder/default/adapter_model.bin -d /content/MagicTime/ckpts/Magic_Weights/magic_text_encoder/default -o adapter_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/spaces/BestWishYsh/MagicTime/raw/main/ckpts/Magic_Weights/magic_text_encoder/default/adapter_config.json -d /content/MagicTime/ckpts/Magic_Weights/magic_text_encoder/default -o adapter_config.json

# %cd /content/MagicTime
# !python inference_magictime.py --config sample_configs/RealisticVision.yaml
# !python inference_magictime.py --config sample_configs/RcnzCartoon.yaml
# !python inference_magictime.py --config sample_configs/ToonYou.yaml

In [ ]:
%cd /content/MagicTime
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, DDIMScheduler
from diffusers.utils.import_utils import is_xformers_available
from utils.unet import UNet3DConditionModel
from utils.pipeline_magictime import MagicTimePipeline
from utils.util import save_videos_grid
from utils.util import load_weights

tokenizer    = CLIPTokenizer.from_pretrained("/content/MagicTime/ckpts/Base_Model/stable-diffusion-v1-5", subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained("/content/MagicTime/ckpts/Base_Model/stable-diffusion-v1-5", subfolder="text_encoder").cuda()
vae          = AutoencoderKL.from_pretrained("/content/MagicTime/ckpts/Base_Model/stable-diffusion-v1-5", subfolder="vae").cuda()
unet = UNet3DConditionModel.from_pretrained_2d("/content/MagicTime/ckpts/Base_Model/stable-diffusion-v1-5", subfolder="unet",
unet_additional_kwargs = {
    "use_inflated_groupnorm": True,
    "use_motion_module": True,
    "motion_module_resolutions": [1, 2, 4, 8],
    "motion_module_mid_block": False,
    "motion_module_type": "Vanilla",
    "motion_module_kwargs": {
        "num_attention_heads": 8,
        "num_transformer_block": 1,
        "attention_block_types": ["Temporal_Self", "Temporal_Self"],
        "temporal_position_encoding": True,
        "temporal_position_encoding_max_len": 32,
        "temporal_attention_dim_div": 1,
        "zero_initialize": True
    },
    "noise_scheduler_kwargs": {
        "beta_start": 0.00085,
        "beta_end": 0.012,
        "beta_schedule": "linear",
        "steps_offset": 1,
        "clip_sample": False
    }
}).cuda()

if is_xformers_available():
    unet.enable_xformers_memory_efficient_attention()

pipeline = MagicTimePipeline(
    vae=vae, text_encoder=text_encoder, tokenizer=tokenizer, unet=unet,
    scheduler=DDIMScheduler(beta_start=0.00085,
                            beta_end=0.012,
                            beta_schedule="linear",
                            steps_offset=1,
                            clip_sample=False)
).to("cuda")

dreambooth = "ToonYou_beta6" # @param ["ToonYou_beta6", "RealisticVisionV60B1_v51VAE", "RcnzCartoon"]

pipeline = load_weights(
    pipeline,
    motion_module_path="/content/MagicTime/ckpts/Base_Model/motion_module/motion_module.ckpt",
    dreambooth_model_path=f"/content/MagicTime/ckpts/DreamBooth/{dreambooth}.safetensors",
    magic_adapter_s_path="/content/MagicTime/ckpts/Magic_Weights/magic_adapter_s/magic_adapter_s.ckpt",
    magic_adapter_t_path="/content/MagicTime/ckpts/Magic_Weights/magic_adapter_t",
    magic_text_encoder_path="/content/MagicTime/ckpts/Magic_Weights/magic_text_encoder",
).to("cuda")

In [ ]:
import torch
random_seed = torch.randint(0, 2 ** 32 - 1, (1,)).item()
torch.manual_seed(torch.randint(0, 2 ** 32 - 1, (1,)).item())

prompt = 'An ice cube is melting.' # @param {type:"string"}
num_inference_steps = 25 # @param {type:"number"}
guidance_scale = 8.5 # @param {type:"number"}
width = 512 # @param {type:"number"}
height = 512 # @param {type:"number"}
video_length = 16 # @param {type:"number"}
sample = pipeline(prompt,
                  num_inference_steps=num_inference_steps,
                  guidance_scale=guidance_scale,
                  width=width,
                  height=height,
                  video_length=video_length,
                  ).videos
save_videos_grid(sample, f"/content/MagicTime/output-{random_seed}.gif")

from PIL import Image
gif = Image.open(f"/content/MagicTime/output-{random_seed}.gif")
gif